In [6]:
import pandas as pd

pop = pd.read_csv('datasets/data-USstates-master/state-population.csv')
areas = pd.read_csv('datasets/data-USstates-master/state-areas.csv')
abbrevs = pd.read_csv('datasets/data-USstates-master/state-abbrevs.csv')

print(pop.head(), '\n'); print(areas.head(), '\n'); print(abbrevs.head(), '\n')

  state/region     ages  year  population
0           AL  under18  2012   1117489.0
1           AL    total  2012   4817528.0
2           AL  under18  2010   1130966.0
3           AL    total  2010   4785570.0
4           AL  under18  2011   1125763.0 

        state  area (sq. mi)
0     Alabama          52423
1      Alaska         656425
2     Arizona         114006
3    Arkansas          53182
4  California         163707 

        state abbreviation
0     Alabama           AL
1      Alaska           AK
2     Arizona           AZ
3    Arkansas           AR
4  California           CA 



In [7]:
merged = pd.merge(pop, abbrevs, how='outer',
                 left_on='state/region', right_on='abbreviation')

merged = merged.drop('abbreviation', 1)
merged.head()

,state/region,ages,year,population,state
0,AL,under18,2012,1117489.0,Alabama
1,AL,total,2012,4817528.0,Alabama
2,AL,under18,2010,1130966.0,Alabama
3,AL,total,2010,4785570.0,Alabama
4,AL,under18,2011,1125763.0,Alabama


In [8]:
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state            True
dtype: bool

In [13]:
merged[merged['population'].isnull()]
# seems all population null values are for peurto rico prior to 2000 - likely data isn't available from original source
# crspndng state also NaN so missing from abbrevs

# find unique missing mapping entries
merged.loc[merged['state'].isnull(), 'state/region'].unique()

# fix by filling appropriate entries
merged.loc[merged['state/region'] == 'PR', 'state'] = 'Peurto Rico'
merged.loc[merged['state/region'] == 'USA', 'state'] = 'United States'

merged.isnull().any()
merged[merged['population'].isnull()]

,state/region,ages,year,population,state
2448,PR,under18,1990,NaN,Peurto Rico
2449,PR,total,1990,NaN,Peurto Rico
2450,PR,total,1991,NaN,Peurto Rico
2451,PR,under18,1991,NaN,Peurto Rico
2452,PR,total,1993,NaN,Peurto Rico
2453,PR,under18,1993,NaN,Peurto Rico
2454,PR,under18,1992,NaN,Peurto Rico
2455,PR,total,1992,NaN,Peurto Rico
2456,PR,under18,1994,NaN,Peurto Rico
2457,PR,total,1994,NaN,Peurto Rico


In [20]:
## merge with the area data
final = pd.merge(merged, areas, on='state', how='left')
final.head()

# again check for nulls to see if there were any mismatches
final.isnull().any()

'''
state/region     False
ages             False
year             False
population        True
state            False
area (sq. mi)     True

null in area col so se ewhich regions were ignored
'''
final['state'][final['area (sq. mi)'].isnull()].unique()  # ['Peurto Rico' 'United States']. But tb only US (page 157)
final.dropna(inplace=True)
#final.head()

['Peurto Rico' 'United States']


In [25]:
# to answer the que, select only 2010 corresponding data
data2010 = final.query("year==2010 & ages=='total'")
data2010.head()

# compute the population density and display it in order.
data2010.set_index('state', inplace=True)
density = data2010['population'] / data2010['area (sq. mi)']

density.sort_values(ascending=False, inplace=True)
print(density.head(), '\n')
'''
tate
District of Columbia    8898.897059
New Jersey              1009.253268
Rhode Island             681.339159
Connecticut              645.600649
Massachusetts            621.815538
dtype: float64 
'''
print(density.tail(), '\n')
'''
state
South Dakota    10.583512
North Dakota     9.537565
Montana          6.736171
Wyoming          5.768079
Alaska           1.087509
dtype: float64 
'''

state
District of Columbia    8898.897059
New Jersey              1009.253268
Rhode Island             681.339159
Connecticut              645.600649
Massachusetts            621.815538
dtype: float64 

state
South Dakota    10.583512
North Dakota     9.537565
Montana          6.736171
Wyoming          5.768079
Alaska           1.087509
dtype: float64 

